# Regression approach
## Valentin Courgeau 17098662 valentin.courgeau.17@ucl.ac.uk
## John Goodacre 13064947 john.goodacre.13@ucl.ac.uk

In [ ]:
import pywt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import numpy.random as rnd
import os
import sys
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization, ConvLSTM2D
from keras.callbacks import EarlyStopping
from keras import backend as K
import time

from functools import partial
from sklearn.metrics import confusion_matrix

%matplotlib inline


### Wavelet Transform

In [ ]:
current_level = 2
current_mode = 'constant'
current_wavelet = pywt.Wavelet('haar')

def save_as_csv(data_array, file_name="data"):
    output_name = file_name + ".csv"
    np.savetxt(output_name, data_array, delimiter=",", fmt='%-7.4f', 
               header=str(data_array.dtype.names).replace('(',"").replace(')',"").replace("#", "") + " ",
              comments='')
    
def apply_wt_once(data, wavelet='haar', level=2, mode='constant'):
    haar_dwt = pywt.wavedec(data, wavelet=wavelet, level=level, mode=mode)
    
    # delete the high frequencies from the decomposition
    haar_dwt[1] = np.zeros_like(haar_dwt[1])
    haar_dwt[2] = np.zeros_like(haar_dwt[2])
  
    return pywt.waverec(haar_dwt, mode=mode, wavelet=wavelet)

def apply_wt_twice(data, wavelet='haar', level=2, mode='constant'):
    wt_results = apply_wt_once(data=data,
                        wavelet=current_wavelet,
                        level=current_level,
                        mode=current_mode)
    
    return apply_wt_once(data=wt_results,
                        wavelet=current_wavelet,
                        level=current_level,
                        mode=current_mode)

def get_accuracy(data1, data2):
    return np.std(data1-data2)

def apply_wt_twice_on_dataset(dataset, wavelet='haar', level=2, mode='constant'):
    col_names = dataset.dtype.names
    results = np.zeros_like(dataset)

    
    for name in col_names:
        #if name not in ignored_col_names:
        temp = apply_wt_twice(
                            dataset[name],
                            wavelet=wavelet,
                            level=level,
                            mode=mode
                             )
        results[name] = temp[:len(dataset[name])]
    return results
 
def end_to_end_twice_wt_with_csv(input_file_name, output_file_name, wavelet='db2', level=2, mode='constant'):
    data_array = np.genfromtxt('data/' + input_file_name, delimiter=',', dtype=float, names=True)
    res = apply_wt_twice_on_dataset(data_array,
                                wavelet=wavelet,
                                level=level,
                                mode=mode)
    save_as_csv(res, file_name="data/data_wt/" + output_file_name)
    res = [list(x) for x in res]
    return np.array(res)




### Prepare Data

In [ ]:

#normalise dataset pre and post autoencoder input
def normalise_dataset(dataset):
    n_cols = dataset.shape[1]
    norm_dataset = np.zeros_like(dataset)
    for i in range(n_cols): 
        mean = np.sum(dataset[:,i])/len(dataset[:,i])
        std_dev = np.dot(dataset[:,i]-mean, dataset[:,i]-mean)/(len(dataset[:,i]) - 1)
        std_dev = np.sqrt(std_dev)
        norm_dataset[:,i] = (dataset[:,i] - mean) / std_dev

    return norm_dataset

# conversion code to create lookbacks for lstms
# convert an array of values into a dataset matrix
def create_dataset(trainX, trainY, look_back=1):
    dataX, dataY = [], []
    for i in range(len(trainX)-look_back-1):
        a = trainX[i:(i+look_back), :]
        dataX.append(a)
        dataY.append(trainY[i + look_back-1])
    return np.array(dataX), np.array(dataY)


#this is to create data set inputs to the mlp or lstm
def make_reg_data(latent_val, y_train, look_back):

    features = latent_val.shape[1]
    all_trainX = latent_val 

    trainX = all_trainX[:round(0.9*len(all_trainX))]
    testX = all_trainX[round(0.9*len(all_trainX)):]

    all_trainY = np.array(y_train[:-1])
    trainY = all_trainY[:round(0.9*len(all_trainY))]
    testY = all_trainY[round(0.9*len(all_trainY)):]
    
    if look_back != 0:
        ##### for an mlp dont do the lookback stuff ######
        trainX, trainY = create_dataset(trainX, trainY, look_back)
        testX, testY = create_dataset(testX, testY, look_back) 
    return trainX, trainY, testX, testY


#examine initial features
def make_raw_data(input_file, features, wavelet, normalise, cheat, cheat_fac):
    #including the creation of the wavelet stuff
    #returns normalised X, and target y, currently hard coded files
    #for targets it assumes a close price for the input file
    #also for now hard coded to data/data_wt
    
    raw_data = pd.read_csv("data/" + input_file)

    #set up with complete cheat features to check it can learn - an MLP does what about lSTM
    cheat_data = raw_data[features].copy() # raw_data[['Ntime', 'time', 'Close Price','Open Price']].copy()
    if cheat:
        cheat_data["Cheat"] = raw_data['Close Price'].shift(-1)
    cheat_data = cheat_data[:-1] #to keep it clean will take the last off whether cheat or not - because of y's
    cheat_data.to_csv('data/cheat.csv', index=False)
    
    #run Valentin's wavelet stuff
    ok = end_to_end_twice_wt_with_csv("cheat.csv", "cheat500_wt")

    if wavelet:
        x_train = np.genfromtxt('data/data_wt/cheat500_wt.csv', delimiter=',', dtype=None, names=True)
    else:
        x_train = np.genfromtxt('data/cheat.csv', delimiter=',', dtype=None, names=True)

    x_train = [list(item) for item in x_train]
    x_train = np.array(x_train)

    #set up target and test y's hard coded target price
    y_lazy = pd.read_csv("data/"+input_file)['Close Price'] 
    y_train = y_lazy.shift(-1)
    
    #because of the shift the last value will be n/a thus...
    x_train = x_train[:-1]
    y_train = y_train[:-1]

    # Normalised dataset
    if normalise:
        x_train = normalise_dataset(x_train)
        
    if cheat:
        x_train[:,-1] = x_train[:,-1] +np.random.randn(x_train.shape[0])*cheat_fac
    
    return x_train, y_train



### Stacked Autoencoders

In [ ]:
#Autoencoder functions

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def train_autoencoder(X_train, n_neurons, n_epochs, batch_size,
                       activation, learning_rate = 0.005, l2_reg = 0.0005,
                      seed=42):
    graph = tf.Graph()
    with graph.as_default():
        tf.set_random_seed(seed)

        n_inputs = X_train.shape[1]

        X = tf.placeholder(tf.float32, shape=[None, n_inputs])
        
        my_dense_layer = partial(
            tf.layers.dense,
            activation=activation,
            kernel_initializer=tf.contrib.layers.variance_scaling_initializer(),
            kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_reg))

        hidden = my_dense_layer(X, n_neurons, name="hidden")
        outputs = my_dense_layer(hidden, n_inputs, activation=None, name="outputs")

        reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))

        reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        loss = tf.add_n([reconstruction_loss] + reg_losses)

        optimizer = tf.train.AdamOptimizer(learning_rate)
        training_op = optimizer.minimize(loss)

        init = tf.global_variables_initializer()

    with tf.Session(graph=graph) as sess:
        init.run()
        for epoch in range(n_epochs):
            n_batches = len(X_train) // batch_size
            for iteration in range(n_batches):
                sys.stdout.flush()
                indices = rnd.permutation(len(X_train))[:batch_size]
                X_batch = X_train[indices]
                sess.run(training_op, feed_dict={X: X_batch})
            loss_train = reconstruction_loss.eval(feed_dict={X: X_batch})
            if epoch % 500 == 0:
                print("\r{}".format(epoch), "Train MSE:", loss_train)
        params = dict([(var.name, var.eval()) for var in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)])
        hidden_val = hidden.eval(feed_dict={X: X_train})
        return hidden_val, params["hidden/kernel:0"], params["hidden/bias:0"], params["outputs/kernel:0"], params["outputs/bias:0"]


def sae(x_train, N_EPOCHS, BATCH_SIZE,LEARNING_RATE, neurons):
    reset_graph()
    ACTIVATION = tf.nn.tanh

    hidden_output1, W1, b1, W8, b8 = train_autoencoder(x_train, n_neurons=neurons[0], 
                                                       n_epochs=N_EPOCHS, 
                                                       batch_size=BATCH_SIZE, 
                                                       learning_rate=LEARNING_RATE,
                                                       activation=ACTIVATION)
    hidden_output2, W2, b2, W7, b7 = train_autoencoder(hidden_output1, n_neurons=neurons[1], 
                                                       n_epochs=N_EPOCHS, 
                                                       batch_size=BATCH_SIZE, 
                                                       learning_rate=LEARNING_RATE,
                                                       activation=ACTIVATION)
    hidden_output3, W3, b3, W6, b6 = train_autoencoder(hidden_output2, n_neurons=neurons[2], 
                                                       n_epochs=N_EPOCHS, 
                                                       batch_size=BATCH_SIZE, 
                                                       learning_rate=LEARNING_RATE,
                                                       activation=ACTIVATION)
    o4, W4, b4, W5, b5 = train_autoencoder(hidden_output3,n_neurons=neurons[3], 
                                                       n_epochs=N_EPOCHS, 
                                                       batch_size=BATCH_SIZE, 
                                                       learning_rate=LEARNING_RATE,
                                                       activation=ACTIVATION)


    #Extract the latent outputs from the stacked ae
    reset_graph()
    activation = ACTIVATION

    n_inputs = 1*(x_train.shape[1])

    X = tf.placeholder(tf.float32, shape=[None, n_inputs])
    
    hidden1 = activation(tf.matmul(X, W1) + b1)
    hidden2 = activation(tf.matmul(hidden1, W2) + b2)
    hidden3 = activation(tf.matmul(hidden2, W3) + b3)
    hidden4 = activation(tf.matmul(hidden3, W4) + b4)
    hidden5 = activation(tf.matmul(hidden4, W5) + b5)
    hidden6 = activation(tf.matmul(hidden5, W6) + b6)
    hidden7 = activation(tf.matmul(hidden6, W7) + b7)
    
    outputs = tf.matmul(hidden7, W8) + b8

    #unnecessary - should be the same as o4 but just to check
    with tf.Session() as sess:
            latent_val = hidden4.eval(feed_dict={X: x_train})

    # Normalise output of SAEs
    latent_val = normalise_dataset(latent_val)

    # Save output in .csv file
    latent_out = pd.DataFrame(latent_val) #latent_val
    latent_out.to_csv('data/auto_out.csv', index=False)
    
    reset_graph()
    
    return latent_val  
    

### Metrics and trading strategy

In [ ]:
### Adding goodness of fit measures
# MAPE
def compute_mape(y, y_pred):
    assert len(y) == len(y_pred)
    return np.sum(np.abs((y-y_pred) / y)) / len(y)

def compute_r(y, y_pred):
    assert len(y) == len(y_pred)
    mean_pred = np.mean(y_pred)
    return  np.dot(y - mean_pred, y_pred - mean_pred) / np.sqrt(np.sum(np.square(y - mean_pred)) * np.sum(np.square(y_pred - mean_pred)))

def compute_theil_u(y, y_pred):
    assert len(y) == len(y_pred)
    
    n = len(y)
    temp = np.sqrt(np.sum(np.square(y)) / n)
    temp += np.sqrt(np.sum(np.square(y_pred)) / n)
    temp = np.sqrt(np.sum(np.square(y - y_pred)) / n) / temp 
    return temp

# Profitability

# Transaction Costs
BUY_COST = 0.05/100
SELL_COST = 0.05/100

def daily_pnl(y_tomorrow, y_today, buy=True, buy_cost=BUY_COST, sell_cost=SELL_COST):
    if(buy):
        return (y_tomorrow - y_today - (sell_cost*y_tomorrow + buy_cost*y_today)) / y_today
    else:
        return (y_today - y_tomorrow - (buy_cost*y_today + sell_cost*y_tomorrow)) / y_today
    

def buy_and_sell(y_truth, y_pred):
    assert len(y_pred) == len(y_truth)
    n = len(y_pred)
    # Vector of 1s for buy, -1s for sell. Approx: no exact predictions...
    buy_sell = [1 if y_pred[index+1] > y_truth[index] else -1 if y_pred[index+1] < y_truth[index] else 0 for index in range(n-1)]
    ret = [daily_pnl(y_truth[index+1], y_truth[index]) if buy_sell[index] == 1 else
           daily_pnl(y_truth[index+1], y_truth[index], buy=False) for index in range(n-1)]
    return ret # np.sum(ret)*100
    

### Running the model

In [ ]:

def run_regression(trainX, trainY, testX, testY, look_back, LEARNING_RATE, BATCH_SIZE, EPOCHS, neurons, dropout, decay=1e-5):
    #reset_graph()
    K.clear_session()
    # create and fit the LSTM network
    
    opti_adam = keras.optimizers.Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay)
    
    features = trainX.shape[1]
    lstm_features = trainX.shape[-1]

    model = Sequential()

    #lstm test
    if look_back != 0:
        for i in enumerate(neurons):
            if i[0] != len(neurons)-1:
                model.add(LSTM(i[1], input_shape=(look_back, lstm_features),  return_sequences=True, unroll=True))
                model.add((Dropout(dropout)))
                model.add(BatchNormalization())
            else:
                model.add(LSTM(i[1]))
                model.add((Dropout(dropout)))
                model.add(BatchNormalization())
    #MLP Test
    else:
        for i in enumerate(neurons): 
            model.add(Dense(i[1], input_dim=features, activation= 'relu'))
            model.add((Dropout(dropout)))

    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=opti_adam)
    
    history = model.fit(trainX, trainY, epochs=EPOCHS, batch_size=BATCH_SIZE,
              verbose=1, callbacks=[plot_losses], validation_split = 0.1)
    
    return model, history

#Function to enable live graph of losses as training progresses 
#note it is called in this cell also and used later
from IPython.display import clear_output

# Live plot for loss during training
# Taken from: https://gist.github.com/stared/dfb4dfaf6d9a8501cd1cc8b8cb806d2e
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        # Reset the plot to only have the last 100 epochs
        if len(self.x) % 100 == 0:
            self.on_train_begin()
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()


def run_set_regressions(params_list, folder_path):
    output_list = []   
    for param in params_list:
        # Start timer
        start_time = time.time()
        if param['lookback'] != 0:
            layers_to_neurons =  param['LSTM_neurons']
        else:
            layers_to_neurons =  param['MLP_neurons']

        #get data - potentially wavelet transformed, and potentially with a cheat and cheat_fac
        x_train, y_train = make_raw_data(param['input_file'], param['features'], param['wavelet'], 
                                         param['normalise'], param['cheat'], param['cheat_fac'])
        #then get the output from the autoencoder
        if param['auto']:
            latent_val = sae(x_train, param['N_EPOCHS'], param['BATCH_SIZE'], 
                             0.005, param['neurons']) #the output is normalised
        else:
            latent_val = x_train #this could be raw data or wavelet output depending on our choice
            latent_val = normalise_dataset(latent_val)

        trainX, trainY, testX, testY = make_reg_data(latent_val, y_train, param['lookback'])   
        
        model, history = run_regression(trainX, trainY, testX, testY, 
                                        param['lookback'], param['LEARNING_RATE'], param['BATCH_SIZE_LSTM'], 
                                        param['EPOCHS'], layers_to_neurons, param['dropout'])
        end_time = start_time - time.time()
        predY = model.predict(testX)[:,0]
        
        model.summary()
        performance_metrics = {'MAPE': compute_mape(testY, predY),
                              'R': compute_r(testY, predY),
                              'theilU': compute_theil_u(testY, predY),
                              'buy_and_sell': buy_and_sell(testY, predY),
                               'preparation_and_training_time':end_time,
                              'testY': np.array(testY),
                              'predY': np.array(predY),
                              'val_loss': history.history['val_loss'],
                              'loss': history.history['loss'],
                              'testX': np.array(testX),
                              'trainX': np.array(trainX),
                              'trainY': np.array(trainY)}
        
        # Saving everything
        model.save(folder_save + param['model_name'] + ".h5")
        
        with open(folder_save + param['model_name'] + ".yml", 'w') as outfile:
            yaml.dump(performance_metrics, outfile, default_flow_style=False)
        
        output_list.append((param, model, history, performance_metrics))
    
    return output_list

### Using the fixture file to run regressions

In [ ]:
# Use YAML fixture file to run examples
import yaml
reg_list = "reg_list_test.yml"
folder_save = "prediction_results/"

with open("data/" + reg_list, 'r') as stream:
    try:
        regression_dict = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

reg_list = []
for reg in regression_dict:
    temp_reg = {}
    for x in regression_dict[reg]:
        temp_reg['model_name'] = reg
        for key, value in x.items():
            temp_reg[key] = value
    reg_list.append(temp_reg)
    
runs = run_set_regressions(reg_list, folder_save)